In [1]:
import pandas as pd
from datetime import datetime
import sql_functions as sql_int

In [2]:
def new_tournament(tournament_add,name,short_name,start,fin):
    start = datetime.strptime(start, '%Y-%m-%d')
    fin = datetime.strptime(fin, '%Y-%m-%d')
    tournament_add.append([name,short_name,start,fin])
def add_tournament(tournament_lst,tournament_df):
    df_add = pd.DataFrame(tournament_lst,columns=['name','short_name','start','fin'])
    df_add.drop_duplicates(inplace=True)
    tournament_df = pd.concat([tournament_df,df_add],ignore_index=True)
    return tournament_df

## New tournament
#### Read new tournaments from CSV-file

In [2]:
tournament_add = []
tournament = pd.read_csv('/program/tft_overview/tft_Tables/data_sheets/tournaments.csv',
                            dtype={'name' : 'str', 'short_name' : 'str' },
                            parse_dates=["start", "fin"])
df_tournament = tournament.copy()
#df_tournament[['name','short_name']] = df_tournament[['name','short_name']].apply(lambda x : x.str.lower())

#### Example for new tournament

In [4]:
new_tournament(tournament_add,
               'Rising Legends Runeterra Reforged - Open Qualifier 2 Day 2',
               'RL RR OQ2 D2',
               '2023-08-06',
               '2023-08-06'
               )

In [5]:
df_tournament = add_tournament(tournament_add,df_tournament)
#df_tournament[['name','short_name']] = df_tournament[['name','short_name']].apply(lambda x : x.str.lower())

## Get existing Tournaments

In [3]:
schema = 'tft_table'
table = 'tournament'
sql_query = f'''
            select * from {table};
            '''

existing_tournament = pd.DataFrame(sql_int.get_data_alchemy(sql_query),
                                  columns=['tournament_id','name','short_name','start','fin'])

df_existing_tournament = existing_tournament.copy()
df_existing_tournament[['start','fin']] = df_existing_tournament[['start','fin']].apply(pd.to_datetime)
df_existing_tournament['existing'] = 'Yes'
#df_existing_tournament[['name','short_name']] = df_existing_tournament[['name','short_name']].apply(lambda x : x.str.lower())
df_tournament = pd.concat([df_existing_tournament,df_tournament],ignore_index=True)

In [4]:
df_tournament[df_tournament.duplicated(keep = False, subset=['name']) == True].sort_values(by=['name','short_name'])

,tournament_id,name,short_name,start,fin,existing
0,1.0,Rising Legends Runeterra Reforged - Open Quali...,RL RR OQ2 D1 GA,2023-08-05,2023-08-05,Yes
8,NaN,Rising Legends Runeterra Reforged - Open Quali...,RL RR OQ2 D1 GA,2023-08-05,2023-08-05,NaN


In [5]:
df_tournament = (df_tournament
                  .drop_duplicates(subset=['name'],keep=False)
                  .query('existing != "Yes"')
                  .drop(['tournament_id','existing'],axis=1)
                  )

In [6]:
df_tournament

,name,short_name,start,fin
1,Rising Legends Runeterra Reforged - Open Quali...,RL RR OQ1 D1 GA,2023-07-08,2023-07-08
2,Rising Legends Runeterra Reforged - Open Quali...,RL RR OQ1 D1 GB,2023-07-08,2023-07-08
3,Rising Legends Runeterra Reforged - Open Quali...,RL RR OQ1 D2,2023-07-09,2023-07-09
4,Rising Legends Runeterra Reforged - Open Quali...,RL RR OQ1 D3,2023-07-10,2023-07-10
5,Rising Legends Runeterra Reforged - Golden Spa...,RL RR GS1 D1,2023-07-14,2023-07-14
6,Rising Legends Runeterra Reforged - Golden Spa...,RL RR GS1 D2,2023-07-15,2023-07-15
7,Rising Legends Runeterra Reforged - Golden Spa...,RL RR GS1 D3,2023-07-16,2023-07-16
9,Rising Legends Runeterra Reforged - Open Quali...,RL RR OQ2 D1 GB,2023-08-05,2023-08-05
10,Rising Legends Runeterra Reforged - Open Quali...,RL RR OQ2 D2,2023-08-06,2023-08-06
11,Rising Legends Runeterra Reforged - Open Quali...,RL RR OQ2 D3,2023-08-07,2023-08-07


push new tournaments into Database

In [7]:
sql_int.push_to_database(df_tournament,table,sql_int.get_engine_alchemy(),schema)

The tournament table was imported successfully.


: 